In [1]:
import os

import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras import models
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, Flatten,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sn
import random

total_files = 0
total_songs = 0

# find total files
for root, dirs, files in os.walk("CAP6610SP24_test_set/Other", topdown=False):
   for name in files:
      filename = os.path.join(root, name)
      if filename.find(".mp3") != -1 or filename.find(".wav") != -1 or filename.find(".flac") != -1 or filename.find(".avi") != -1 or filename.find(".m4a") != -1 or filename.find(".ogg") != -1 :
          total_files = total_files + 1
          total_songs += 1 


# --------Extract data from csv file--------------------------------------

data = pd.read_csv('other_test_features_10.csv')
data.head()            
new_files = data['filename']
new_files = list(new_files)
for i in range(len(new_files)) :
    new_files[i] = (new_files[i].split("chunk") )[0]


data = data.drop(['filename'],axis=1)
data = data.drop(['mfcc14'],axis=1)
data = data.drop(['mfcc19'],axis=1)
data = data.drop(['mfcc9'],axis=1)
data = data.drop(['mfcc12'],axis=1)
data = data.drop(['mfcc20'],axis=1)
data = data.drop(['chroma_stft'],axis=1)

data = np.asarray(data)

X = data


X = np.reshape(X, (X.shape[0],1, X.shape[1]))

print("X_train shape ",X.shape)

model = load_model('ModelBest-7481.h5')

print("\nTesting ...")

# Predict using saved model

X= X.astype('float32')

predictions = model.predict(X)
print(predictions)
row,col = predictions.shape
count = 0
for i in range(row) :
    # true_val = np.argmax(y_test[i])
    predicted_val = np.argmax(predictions[i]) 
    print(new_files[i],"    got ",predicted_val)
    count += 1
print("Total files ",count)


new_predicted_ind = []
i = 0

ind = []     
start = 0
for i in range(len(new_files)) :
    if i == 0 :
        start = 0
    elif i == len(new_files) - 1 :
        if new_files[i] == new_files[i-1] :
            ind.append([new_files[i],start,len(new_files)-1])  
        else :
            ind.append([new_files[i],len(new_files)-1,len(new_files)-1])  

    else :
         if new_files[i] != new_files[i-1] :
             end = i-1
             ind.append([new_files[i-1],start,end])  
             start = i
    
print("indices calculated") 
print("-----------------------------------------------------------------------------") 
print("prog = 0 || nonprog = 1") 

for i in range(row) :
    new_predicted_ind.append(np.argmax(predictions[i]))

# Find majority result for song chunks

def calc_majority(indices,predicted) :
    name = indices[0]
    start = indices[1]
    end = indices[2]
    new_predicted = predicted[start:end+1]
    maj_pred = 0
    for i in range(len(new_predicted)) :
        if new_predicted[i] == 0 :
            maj_pred += 1
    if maj_pred <= 2*len(new_predicted)/3  :       
       ret_pred_val = 1
    else:
        ret_pred_val=0       
    return [ret_pred_val, name]



majority_pred = []
majority_name = []


# Find accuracy for all chunks by checking majority output
for i in range(len(ind)) :
    ret_pred_val, name = calc_majority(ind[i],new_predicted_ind)
    majority_pred.append(ret_pred_val)
    majority_name.append(name)
count = 0
for i in range(len(majority_pred)) :
    print(majority_name[i],"-------------- predicted",majority_pred[i])

X_train shape  (635, 1, 20)

Testing ...
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
[[0.69553244 0.32466763]
 [0.68966836 0.32926434]
 [0.6959747  0.32432106]
 ...
 [0.69589484 0.32438368]
 [0.69598037 0.3243166 ]
 [0.69371414 0.32609275]]
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0
b1-master_boot_record-dma_4_cascade.mp3     got  0